# StyleAdv attack

In [ ]:
!git clone https://github.com/yifengd/StyleAttack.git
!pip install transformers >> /dev/null


Cloning into 'StyleAttack'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 370 (delta 10), reused 19 (delta 7), pack-reused 348
Receiving objects: 100% (370/370), 58.27 MiB | 23.88 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
!git -C /content/StyleAttack pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/yifengd/StyleAttack
   c81a687..2236dc3  main       -> origin/main
Updating c81a687..2236dc3
Fast-forward
 data/clean/ag/test_1000.tsv | 1000 +++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 1000 insertions(+)
 create mode 100644 data/clean/ag/test_1000.tsv


In [ ]:
# Mount on Google Drive File System
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/StyleAttack/experiments
!CUDA_VISIBLE_DEVICES=0 python attack.py --model_name  textattack/distilbert-base-uncased-ag-news --bert_type distilbert-base-uncased --orig_file_path ../data/clean/ag/test_1000.tsv --model_dir /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large --output_file_path /content/drive/MyDrive/record_1000.log


/content/StyleAttack/experiments
Some weights of the model checkpoint at /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.bias', 'transformer.extra_embedding_project.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0% 0/999 [00:00<?, ?it/s]/content/StyleAttack/experiments/style_paraphrase/inference_utils.py:95: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray w

In [ ]:
import pandas as pd 
tsv_file = '/content/drive/MyDrive/record_1000.log'
csv_table = pd.read_table(tsv_file,sep='\t')
csv_table.columns

Index(['p_val ', ' orig_sent ', ' adv_sent ', ' labels'], dtype='object')

In [ ]:
csv_table = csv_table.rename(columns={" orig_sent ": "original_text", " adv_sent ": "adversarial_text"})
csv_table.columns

Index(['p_val ', 'original_text', 'adversarial_text', ' labels'], dtype='object')

In [ ]:
csv_table.to_csv('/content/drive/MyDrive/Colab Notebooks/ExplainableAI/ag-news_1000_styleadv_distilbert.csv', index=False)